<a href="https://colab.research.google.com/github/pedrohortencio/cancer-detection/blob/main/PCam/PCam_Model_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teste do Modelo Proposto
## Detecção Histopatológica de Câncer Utilizando Aprendizado por Transferência

![](https://raw.githubusercontent.com/basveeling/pcam/master/pcam.jpg)

>## Universidade Federal de Goiás
>
>### Alunos:
>
>>* Gustavo M. R. 
>>
>>* Iago S. S.
>>
>>* Pedro H. M. R.




####Fonte dos Dados Utilizados:
<blockquote>

[Detecção Histopatológica de Câncer - Kaggle](https://www.kaggle.com/c/histopathologic-cancer-detection/data)

[PatchCamelyon (PCam) Dataset](https://github.com/basveeling/pcam)

</blockquote>
<hr>

In [ ]:
#@title Rode essa célula.
#@markdown _Atente-se às instruções. Duplo clique exibe o código._

print("Iniciando")

### Importação de bibliotecas
from IPython.utils import io
import urllib, zipfile, os, cv2
import urllib.request
from google.colab import files

from tensorflow import keras
import tensorflow as tf
from keras.preprocessing import image
import numpy as np

### Desabilita warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

### Download do modelo
url_model = "https://github.com/pedrohortencio/cancer-detection/raw/main/PCam/model-120k-30052021.zip"
_, _ = urllib.request.urlretrieve(url_model, filename="/content/model.zip")

### Descompactação do modelo
with zipfile.ZipFile('/content/model.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/model')

### Criação do modelo (Sem output)
with io.capture_output() as captured:
    model = keras.models.load_model('/content/model/model')


### Definição da função que baixa as imagens
def download_img(url):
    #url = 'https://www.researchgate.net/profile/Niall-Davis/publication/256541194/figure/fig2/AS:616371377434629@1523965796090/Histology-showing-superficial-spreading-component-of-malignant-melanoma.png' 
    #url = 'https://images-na.ssl-images-amazon.com/images/I/81XlNqHrx5L._AC_SL1200_.jpg'
    #url = 'https://www.webpathology.com/slides-13/slides/Ovary_BrennerTumor13_Malignant.jpg'
    
    ### Extensão da imagem e criação do caminho
    extensao = url.split(".")[-1]
    filename = f"/content/img_original.{extensao}"

    ### Tenta baixar, tanto pelo urllib quanto pelo wget
    try:
        _, _ = urllib.request.urlretrieve(url, filename=filename)
    except:
        try:
            !wget $url -O $filename
        except:
            raise Exception("Não foi possível realizar o download. Verifique se o link termina com a extensão do arquivo (.jpg, .png).")

    ### Tendo sucesso, a imagem é carregada para a memória
    try:
        img = image.load_img(filename, target_size=(96, 96, 3))
    except:
        raise Exception("\nNão foi possível carregar a imagem. Verifique se o link termina com a extensão do arquivo (.jpg, .png).")
    
    ### Retorna-se a imagem e o caminho
    return img, filename


### Lê a imagem a partir de um caminho
def ler_imagem():

    ### Tenta abrir a box para upload (nem sempre funciona por causa dos navegadores)
    try:
        uploaded = files.upload()
    except:
        erro = """\nNão foi possível realizar o upload. Verifique os cookies.
Alternativamente, clique no símbolo de pasta (à esquerda), realize o upload do arquivo.
Clique nos três pontinhos e em 'copiar caminho'. Cole aqui. Ou apenas digite 0 para encerrar.
Caminho:\n
""" 
        ### Solicita o caminho da imagem no google colab em caso de erro
        path = input(erro)

        ### Lê a imagem a partir do caminho
        if path == '0':
            raise Exception("Programa encerrado")   # Exceção caso não seja fornecido
        else:
            try:
                img = image.load_img(path, target_size=(96, 96, 3))
            except:
                raise Exception("\nNão foi possível carregar a imagem. Verifique se o link termina com a extensão do arquivo (.jpg, .png).")
    
            return img, path

    ### Caso o upload dê certo
    for fn in uploaded.keys():
        filename = fn

    filename = f"/content/{filename}"

    try:
        img = image.load_img(filename, target_size=(96, 96, 3))
    except:
        raise Exception("\nNão foi possível carregar a imagem. Verifique se o link termina com a extensão do arquivo (.jpg, .png).")
    
    return img, filename


print("--"*20)
print("Legenda:")
print("\tClasse 0: Sem tumor")
print("\tClasse 1: Com tumor")
print("--"*20)

flag = True         # Encerra o loop
while flag:

    select = -1
    while select != 1 and select != 2:
        try:
            select = int(input("\nFonte da Imagem: 1 - URL ou 2 - Arquivo 0 - Encerrar \n"))
            if select == 0:
                flag = False
                break
        except:
            pass

    if flag == False:
        break

    if select == 1:
        url = input("\nInsira a URL da imagem. O fim da URL DEVE conter a extensão da imagem (.png, .jpg, etc).\n")
        with io.capture_output() as captured:
            img, filename = download_img(url)

    if select == 2:
        print("\nFaça o upload da imagem abaixo:")
        with io.capture_output() as captured:
            img, filename = ler_imagem()


    # Converte a imagem para um NumPy array
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)


    im1 = cv2.imread(filename)

    #print("--"*20)
    print("--"*8, "Predição", "--"*8)
    print(f"Classe 0 (Não há tumor): {model.predict(x)[0][0]: .12f} \nClasse 1 (Há tumor): {model.predict(x)[0][1]: .12f}")

    predicted = np.argmax(model.predict(x), axis=-1)
    print("--"*20)
    print(f"Classe predita: {predicted[0]}")
    print("--"*40)
    #os.remove(filename)